# Enriching Filtered Events

In this notebook, we'll enrich the filtered events from the previous notebook with additional information. We'll use a combination of techniques to enrich the events:

1. Topic modeling using a Large Language Model (LLM) to extract topics from the posts
2. Creating embeddings for semantic search using a transformer model
3. Storing the enriched events in Redis for querying



Embeddings are vector representations of text that capture semantic meaning. They allow us to perform semantic search, which is a search based on meaning rather than exact keyword matching. In this notebook, we'll create embeddings for posts and store them in Redis for later querying.


## Recreating helping functions

In [19]:
@file:DependsOn("redis.clients:jedis:6.0.0")
%use coroutines

In [24]:
import redis.clients.jedis.resps.StreamEntry
import redis.clients.jedis.search.Document

data class Event(
    val did: String,
    val rkey: String,
    val text: String,
    val timeUs: String,
    val operation: String,
    val uri: String,
    val parentUri: String,
    val rootUri: String,
    val langs: List<String>,
    val similarityScore: Double
) {
    companion object {
        fun fromMap(entry: StreamEntry): Event {
            return fromMap(entry.fields)
        }

        fun fromMap(document: Document): Event {
            val fields = document.properties.associate { entry ->  entry.key to entry.value.toString()}
            return fromMap(fields)
        }

        fun fromMap(fields: Map<String, String>): Event {
            return Event(
                did = fields["did"] ?: "",
                rkey = fields["rkey"] ?: "",
                text = fields["text"] ?: "",
                timeUs = fields["timeUs"] ?: "",
                operation = fields["operation"] ?: "",
                uri = fields["uri"] ?: "",
                parentUri = fields["parentUri"] ?: "",
                rootUri = fields["rootUri"] ?: "",
                langs = fields["langs"]?.replace("[", "")?.replace("]", "")?.split(", ") ?: emptyList(),
                similarityScore = fields["similarityScore"]?.toDouble() ?: 0.0
            )
        }
    }
}

In [25]:
import redis.clients.jedis.params.XReadGroupParams
import redis.clients.jedis.StreamEntryID
import kotlinx.coroutines.*
import redis.clients.jedis.JedisPooled
import redis.clients.jedis.bloom.BFReserveParams
import redis.clients.jedis.exceptions.JedisDataException
import redis.clients.jedis.Connection
import redis.clients.jedis.JedisPool
import redis.clients.jedis.Transaction

val jedis = JedisPooled()

fun createConsumerGroup(streamName: String, consumerGroupName: String) {
    try {
        jedis.xgroupCreate(streamName, consumerGroupName, StreamEntryID("0-0"), true)
    } catch (e: Exception) {
        println("Group already exists")
    }
}

fun readFromStream(streamName: String, consumerGroup: String, consumer: String, count: Int): List<Map.Entry<String, List<StreamEntry>>> {
    return jedis.xreadGroup(
        consumerGroup,
        consumer,
        XReadGroupParams().count(count).block(1),
        mapOf(
            streamName to StreamEntryID.XREADGROUP_UNDELIVERED_ENTRY
        )
    ) ?: emptyList()
}

fun consumeStream(
    streamName: String,
    consumerGroup: String,
    consumer: String,
    handlers: List<(Event) -> Pair<Boolean, String>>,
    ackFunction: ((String, String, StreamEntry) -> Unit),
    count: Int = 5,
    limit: Int = 5
) {
    var lastMessageTime = System.currentTimeMillis()
    var consumed = 0

    while (consumed < limit) {
        val entries = readFromStream(streamName, consumerGroup, consumer, count)
        val allEntries = entries.flatMap { it.value }
        allEntries.map { entry ->
            consumed++
            val event = Event.fromMap(entry)

            for (handler in handlers) {
                val (shouldContinue, message) = handler(event)
                ackFunction(streamName, consumerGroup, entry)

                if (!shouldContinue) {
                    println("$consumer: Handler stopped processing: $message")
                    break
                }
            }
        }

        if (allEntries.isEmpty()) {
            val now = System.currentTimeMillis()
            if (now - lastMessageTime >= 2_000) {
                println("$consumer: No new messages for 2 seconds. Stopping.")
                break
            }
        }
    }
}

val printUri: (Event) -> Pair<Boolean, String> = {
    println("Got event from ${it.uri}")
    Pair(true, "OK")
}

fun createBloomFilter(name: String) {
    runCatching {
        jedis.bfReserve(name, 0.01, 1_000_000L, BFReserveParams().expansion(2))
    }.onFailure {
        println("Bloom filter already exists")
    }
}

fun deduplicate(bloomFilter: String): (Event) -> Pair<Boolean, String> {
    return { event ->
        if (jedis.bfExists(bloomFilter, event.uri)) {
            Pair(false, "${event.uri} already processed")
        } else {
            Pair(true, "OK")
        }
    }
}

val jedisPool = JedisPool()

fun ackAndBfFn(bloomFilter: String):  (String, String, StreamEntry) -> Unit = { streamName, consumerGroup, entry ->
    jedisPool.resource.use { jedis ->
        // Create a transaction
        val multi = jedis.multi()

        // Acknowledge the message
        multi.xack(
            streamName,
            consumerGroup,
            entry.id
        )

        // Add the URI to the bloom filter
        multi.bfAdd(bloomFilter, Event.fromMap(entry).uri)

        // Execute the transaction
        multi.exec()
    }
}

## Topic Modeling with Large Language Models
Topic modeling is a technique used to discover abstract topics in a collection of documents. In this notebook, we'll use a Large Language Model to extract topics from posts. This will allow us to categorize posts and make them more searchable.

### Setting Up the Ollama API Client
We'll use the Spring AI Ollama client to interact with the Ollama API.

Ollama is a tool that allows us to run large language models locally.

In [1]:
@file:DependsOn("org.springframework.ai:spring-ai-ollama:1.0.0-RC1")

The prompt we'll use for the LLM is designed to extract software-related topics from posts. The prompt includes examples of how to format the output and what types of topics to include.

In [2]:
val systemPrompt = """
You are a topic classifier specialized in software engineering. Given a post, extract only software-related topics—both explicitly mentioned and reasonably implied.

If a post mentions a tool, language, or library, infer related technologies or domains. For example, if the post mentions LangChain, you may infer topics like “Python”, “AI
”, and “Machine Learning”. Avoid generic terms like “announcement”, “event”, or “release”. Only return the technical topics. Also avoid too narrow topics such as a specific method or command.

If the topic or a very similar is already in the provided list of existing topics, use the one from the list, otherwise, feel free to create a new one.

Format your response as comma separated values (ALWAYS, I MEAN IT):
"topic1, topic2, topic3"

Examples:

Post:
Kotlin is the best programming language for beginners
Output:
"Kotlin, Programming Languages"

Post:
Excited to try some Hugging Face Models with DJL!
Output:
"Hugging Face, Deep Java Library (DJL), Machine Learning, AI, Python, Java"

Post:
Just deployed a FastAPI app using Redis as a cache layer
Output:
"FastAPI, Redis, Python, Web Development, Caching"

Post:
The new version of LangChain is now available!! It’s finally GA!
Output:
"LangChain, Python, AI, Machine Learning"

Post:
Redis is so cool! I love the LOLWUT command.
Output:
"Redis, Database"
"""

Create the Ollama Chat Model

In [3]:
import org.springframework.ai.ollama.OllamaChatModel
import org.springframework.ai.ollama.api.OllamaApi
import org.springframework.ai.ollama.api.OllamaApi.ChatRequest
import org.springframework.ai.ollama.api.OllamaApi.Message
import org.springframework.ai.ollama.api.OllamaApi.Message.Role
import org.springframework.ai.ollama.api.OllamaOptions

val ollamaApi = OllamaApi.builder()
    .baseUrl("http://localhost:11434")
    .build()

val ollamaOptions = OllamaOptions.builder().model("deepseek-coder-v2").build()

val ollamaChatModel = OllamaChatModel.builder()
    .ollamaApi(ollamaApi)
    .defaultOptions(ollamaOptions)
    .build()

### Creating a Topic Modeling Function
This function takes a post as input and uses the Ollama API to extract topics from the post. The function returns a string of comma-separated topics.

In [4]:
import org.springframework.ai.chat.messages.SystemMessage
import org.springframework.ai.chat.messages.UserMessage
import org.springframework.ai.chat.prompt.Prompt

fun topicModeling(post: String, existingTopics: String): String {
    // Build a chat message
    val messages = listOf(
        SystemMessage(systemPrompt),
        UserMessage("Existing topics: $existingTopics"),
        UserMessage("Post: $post")
    )

    val response = ollamaChatModel.call(Prompt(messages))
    return response.result.output.text
}

In [17]:
topicModeling("Kotlin is the best programming languagues for beginners.", "Java, Python, Machine Learning")

 "Kotlin, Programming Languages"

### Talking about Count-min Sketch

Count-min sketch is a probabilistic data structure used for estimating the frequency of events in a stream of data.

It is particularly useful for counting the number of occurrences of items in a large dataset without storing all the items explicitly.

In [11]:
import java.time.LocalDateTime

fun createCountMinSketch(): String {
    val windowBucket = LocalDateTime.now().withSecond(0).withNano(0)
    try {
        jedis.cmsInitByDim("topics-cms:$windowBucket", 3000, 10)
    } catch (e: JedisDataException) {
        println("Count-min sketch already exists")
    }

    return "topics-cms:$windowBucket"
}

### Creating a Topic Extraction Handler
This function creates a handler that extracts topics from an event's text and stores them in Redis. The topics are stored as a pipe-separated string in the "topics" field of the event's hash.

In [13]:
val extractTopics: (Event) -> Pair<Boolean, String> = { event ->
    val existingTopics = jedis.smembers("topics")
    val topics = topicModeling(event.text, existingTopics.joinToString(", "))
        .replace("\"", "")
        .replace("“", "")
        .replace("”", "")
        .split(",")
        .map { it.trim() }

    val cmsKey = createCountMinSketch()
    val multi = jedis.multi()
    multi.cmsIncrBy(cmsKey, topics.associate { it to 1L })
    multi.hset("post:" + event.uri, mapOf("topics" to topics.joinToString("|")))
    multi.sadd("topics", *topics.toTypedArray())
    multi.exec()
    Pair(true, "OK")
}

In [14]:
createConsumerGroup("filtered-events", "topic-extraction-example")

In [16]:
val bloomFilterName = "topic-extraction-bf"
createBloomFilter(bloomFilterName)

In [34]:
runBlocking {
    consumeStream(
        streamName = "filtered-events",
        consumerGroup = "topic-extraction-example",
        consumer = "topic-extraction-1",
        handlers = listOf(deduplicate(bloomFilterName), printUri, extractTopics),
        ackFunction = ackAndBfFn(bloomFilterName),
        count = 100,
        limit = 100
    )
}

Got event from at://did:plc:djglyf5epph2tc44lnqsdvfi/app.bsky.feed.post/3lpcmm7wkrc2y
Got event from at://did:plc:ug7ad2kiztx2jhpuw2w44sjz/app.bsky.feed.post/3lpcmm6hkqc27
Count-min sketch already exists
Got event from at://did:plc:ger7kxkktaqoa32b5lawevrb/app.bsky.feed.post/3lpcmlzriy22k
Count-min sketch already exists
topic-extraction-1: No new messages for 2 seconds. Stopping.


## Creating Embeddings for Semantic Search
In this section, we'll create embeddings for posts. Embeddings are vector representations of text that capture semantic meaning. They allow us to perform semantic search, which is a search based on meaning rather than exact keyword matching.

For example, if I search for:

"Redis is a cool db for Python devs"

I can still match:

"Redis is a great database for Python developers"

### Setting Up the Embedding Model
We'll use the Spring AI Transformers library to create embeddings for posts. This library provides a simple API for creating embeddings using transformer models.

In [18]:
@file:DependsOn("org.springframework.ai:spring-ai-transformers:1.0.0-RC1")
@file:DependsOn("ai.djl.huggingface:tokenizers:0.33.0")

In [19]:
import org.springframework.ai.transformers.TransformersEmbeddingModel

val embeddingModel = TransformersEmbeddingModel()
embeddingModel.afterPropertiesSet()

### Creating an Embedding Handler
This function creates a handler that generates embeddings for an event's text and stores them in Redis. The embeddings are stored as binary data in the "textEmbedding" field of the event's hash.

In [20]:
import java.lang.Float
import java.nio.ByteBuffer
import java.nio.ByteOrder

fun createEmbedding(input: String): ByteArray {
    val embedding = embeddingModel.embed(input)
    val embeddingBytes = ByteArray(Float.BYTES * embedding.size)
    ByteBuffer.wrap(embeddingBytes).order(ByteOrder.LITTLE_ENDIAN).asFloatBuffer().put(embedding)
    return embeddingBytes
}

In [21]:
val createEmbedding: (Event) -> Pair<Boolean, String> = { event ->
    val embeddingBytes = createEmbedding(event.text)
    jedis.hset(("post:" + event.uri).encodeToByteArray(), mapOf("textEmbedding".encodeToByteArray() to embeddingBytes))
    Pair(true, "OK")
}

In [22]:
createConsumerGroup("filtered-events", "embedding-example")

In [23]:
val bloomFilterName = "embedding-bf"
createBloomFilter(bloomFilterName)

In [35]:
runBlocking {
    consumeStream(
        streamName = "filtered-events",
        consumerGroup = "embedding-example",
        consumer = "embedding-1",
        handlers = listOf(deduplicate(bloomFilterName), printUri, createEmbedding),
        ackFunction = ackAndBfFn(bloomFilterName),
        count = 100,
        limit = 100
    )
}

embedding-1: Handler stopped processing: at://did:plc:djglyf5epph2tc44lnqsdvfi/app.bsky.feed.post/3lpcmm7wkrc2y already processed
embedding-1: Handler stopped processing: at://did:plc:ug7ad2kiztx2jhpuw2w44sjz/app.bsky.feed.post/3lpcmm6hkqc27 already processed
embedding-1: Handler stopped processing: at://did:plc:ger7kxkktaqoa32b5lawevrb/app.bsky.feed.post/3lpcmlzriy22k already processed
embedding-1: No new messages for 2 seconds. Stopping.


## Creating a Redis Search Index
In this section, we'll create a Redis Search index to make the enriched events searchable. Redis Search is a module that adds full-text search capabilities to Redis. It allows us to search for events based on their text, topics, and other fields.

### Creating the Index Schema in Code
Now we'll create the index schema in code. We'll use the Jedis client to create the schema and the index.

The following schema defines the fields that will be indexed. The schema includes:
- Text fields for full-text search
- Tag fields for exact matching
- Vector fields for semantic search

```
FT.CREATE postIdx ON HASH PREFIX 1 post: SCHEMA
        parentUri     TEXT
        topics        TAG SEPARATOR "|"
        time_us       TEXT
        langs         TAG
        uri           TEXT
        operation     TAG
        did           TAG
        timeUs        NUMERIC
        rkey          TAG
        textEmbedding VECTOR HNSW 6
            DIM 384
            TYPE FLOAT32
            DISTANCE_METRIC COSINE
        rootUri       TEXT
        text          TEXT
```

In [26]:
import redis.clients.jedis.search.IndexDefinition
import redis.clients.jedis.search.IndexOptions
import redis.clients.jedis.search.Schema
import redis.clients.jedis.search.schemafields.VectorField.VectorAlgorithm

val schema = Schema()
    .addTextField("parentUri", 1.0)
    .addTagField("topics", "|")
    .addTextField("time_us", 1.0)
    .addTagField("langs")
    .addTextField("uri", 1.0)
    .addTagField("operation")
    .addTagField("did")
    .addNumericField("timeUs")
    .addTagField("rkey")
    .addHNSWVectorField(
        "textEmbedding",
        mapOf(
            "type" to "FLOAT32",
            "dim" to "384",
            "distance_metric" to "COSINE",
        )
    )
    .addTextField("rootUri", 1.0)
    .addTextField("text", 1.0)

// Define index options (e.g., prefix)
val rule = IndexDefinition()
    .setPrefixes("post:")

// Create the index
try {
    jedis.ftCreate("postIdx", IndexOptions.defaultOptions().setDefinition(rule), schema)
} catch (e: JedisDataException) {
    println("Index already exists")
}

### Searching the Index
Now that we have created the index, we can search for events based on their topics, text, and other fields. In this example, we'll search for events with the topic "Samba".

Redis Search uses a query language similar to SQL. For example, to search for events with the topic "machine_learning", we would use the query `@topics:{machine_learning}`.

Exact Matching Search

In [37]:
//FT.SEARCH postIdx "@topics:{machine_learning}"
val result = jedis.ftSearch(
    "postIdx",
    "@topics:{Samba}"
)

result.documents.forEach { post ->
    println(Event.fromMap(post))
}

Full Text Search

In [38]:
//FT.SEARCH postIdx "@text:Open source"
val result = jedis.ftSearch(
    "postIdx",
    "@text:Open source"
)

result.documents.forEach { post ->
    println(Event.fromMap(post))
}

Vector Similarity Search

In [39]:
import redis.clients.jedis.search.FTSearchParams
import redis.clients.jedis.search.Query

val vector: ByteArray = createEmbedding("How did they do multiligual TTS?")

val queryString = ("* =>[KNN \$K @textEmbedding \$BLOB AS similarityScore]")

val params = mapOf("BLOB" to vector)

var query = Query(queryString)
    .addParam("K", 1)
    .addParam("BLOB", vector)
    .returnFields("uri", "text", "similarityScore")
    .setSortBy("similarityScore", true)
    .dialect(2)

val result = jedis.ftSearch(
    "postIdx",
    query
)

result.documents.forEach { doc ->
    println(Event.fromMap(doc).toString() + "\n")
}

Pre filtering

In [40]:
//FT.SEARCH postIdx "@text:Open source" "@tag:{Samba}" "*=>[KNN 1 @textEmbedding $BLOB AS similarityScore]"
val vector: ByteArray = createEmbedding("Open source is wizardry stuff")

val queryString = "@topics:{Samba} =>[KNN \$K @textEmbedding \$BLOB AS similarityScore]"

val params = mapOf("BLOB" to vector)

var query = Query(queryString)
    .addParam("K", 5) // Top 5 results (if enough from pre filtering)
    .addParam("BLOB", vector)
    .returnFields("uri", "text", "similarityScore")
    .setSortBy("similarityScore", true)
    .dialect(2)

val result = jedis.ftSearch(
    "postIdx",
    query
)

println(result.totalResults)
result.documents.forEach { doc ->
    println(Event.fromMap(doc).toString() + "\n")
}

0
